In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Sailaja Senapati\\OneDrive\\Desktop\\DEEPDERM\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'c:\\Users\\Sailaja Senapati\\OneDrive\\Desktop\\DEEPDERM\\Kidney-Disease-Classification-Deep-Learning-Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [18]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None  # Initialize model attribute
        self.optimizer = None 
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        self.optimizer = tf.keras.optimizers.Adam()

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.get_base_model()
        self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy', metrics=['accuracy'])

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [19]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e


[2024-05-02 21:31:07,702:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-05-02 21:31:07,706:INFO:common:yaml file: params.yaml loaded successfully]
[2024-05-02 21:31:07,706:INFO:common:created directory at: artifacts]
[2024-05-02 21:31:07,709:INFO:common:created directory at: artifacts\training]
[2024-05-02 21:31:08,888:WARNING:saving_utils:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 527 images belonging to 2 classes.
Found 2110 images belonging to 2 classes.
[2024-05-02 21:31:09,164:WARNING:saving_utils:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Epoch 1/10


c:\Users\Sailaja Senapati\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


65/65 ━━━━━━━━━━━━━━━━━━━━ 216s 3s/step - accuracy: 0.6986 - loss: 0.6395 - val_accuracy: 0.7754 - val_loss: 0.4414
Epoch 2/10
 1/65 ━━━━━━━━━━━━━━━━━━━━ 2:45 3s/step - accuracy: 0.8438 - loss: 0.3251

c:\Users\Sailaja Senapati\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.8438 - loss: 0.3251 - val_accuracy: 0.7333 - val_loss: 0.6885
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 208s 3s/step - accuracy: 0.8055 - loss: 0.4223 - val_accuracy: 0.8184 - val_loss: 0.4197
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8125 - loss: 0.3313 - val_accuracy: 0.8667 - val_loss: 0.4955
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 209s 3s/step - accuracy: 0.8303 - loss: 0.3747 - val_accuracy: 0.8301 - val_loss: 0.4034
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.8125 - loss: 0.3748 - val_accuracy: 0.8667 - val_loss: 0.4542
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 210s 3s/step - accuracy: 0.8242 - loss: 0.3741 - val_accuracy: 0.8301 - val_loss: 0.4082
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.8750 - loss: 0.3533 - val_accuracy: 0.9333 - val_loss: 0.3773
Epoch 9/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 208s 3s/step - accuracy: 0.8281 - loss: 0.3729 - val_accuracy: 0.8301 - val_loss: 0.